In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scvi
import scanpy as sc

sc.set_figure_params(figsize=(4, 4))

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

Global seed set to 0


In [5]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.4 scipy==1.9.3 pandas==1.5.1 scikit-learn==1.1.3 statsmodels==0.13.5 python-igraph==0.10.2 pynndescent==0.5.8


In [16]:
# Import gene matrix, protein matrix, and list of genes for ensembl translation
import anndata as ad
gene_matrix_df = pd.read_csv("https://www.dropbox.com/s/910hhahxsbd9ofs/gene_matrix_all_new.csv?dl=1", header=0, sep=",")
# print(gene_matrix_df)

protein_df = pd.read_csv("https://www.dropbox.com/s/xzobh2p13cqt4y5/protein_expr_all.csv?dl=1", header=0, sep=",")
# print(protein_df)

genes_list_df = pd.read_csv("https://www.dropbox.com/s/11dmp8ki6tui2zu/genes.tsv?dl=1", header=None, sep="\t")
# print(genes_list_df)

# Change first column of protein matrix to not be the index
protein_df.reset_index(inplace=True)
# print(protein_df.iloc[:,0])

In [18]:
# Removing ADT_ prefix from proteins (colname function in R)
import re
def clean_adt(cite_protein_df):
    cite_protein_df_copy = cite_protein_df.copy(deep=True)
    cite_protein_df_copy.iloc[:,0] = cite_protein_df_copy.iloc[:,0].apply(lambda x: re.sub("ADT_", "", x))
    return cite_protein_df_copy

# test = clean_adt(protein_df)
# print(test)

      index  m1_AAACCTGAGAGCAATT.1  m1_AAACCTGCAAGCTGTT.1  \
0      B220                   1072                    627   
1     CD106                     10                      2   
2     CD11b                     12                      5   
3     CD11c                      5                      4   
4   CD16.32                    118                     55   
5     CD169                      6                      0   
6      CD19                     59                     44   
7   CD21.35                    163                     60   
8      CD27                      4                      1   
9       CD3                      4                      0   
10     CD31                     46                     26   
11     CD35                     43                     22   
12      CD4                     13                      4   
13     CD44                     34                     29   
14     CD45                    719                    342   
15      CD5             

In [19]:
# Convert ensembl names to gene names
def convert_ensembl(gene_matrix_df, genes_list_df):
    # Copy of gene matrix
    gene_matrix_df_copy = gene_matrix_df.copy(deep=True)
    
    # Transpose
    gene_matrix_df_copy_transposed = gene_matrix_df_copy.T
    
    # Create dictionary from gene list for translating ensembl names
    genes_list_dict = dict(zip(genes_list_df[0], genes_list_df[1]))
    
    # Return a new matrix
    translated_matrix = gene_matrix_df_copy_transposed.rename(columns = lambda x: genes_list_dict.get(x))
    return translated_matrix

In [24]:
# Translate ensembl names
converted_matrix = convert_ensembl(gene_matrix_df, genes_list_df)
print(converted_matrix)

                       Mrpl15  Lypla1  Tcea1  Atp6v1h  Rb1cc1  4732440D04Rik  \
m1_AAACCTGAGAGCAATT.1       0       0      0        0       0              0   
m1_AAACCTGCAAGCTGTT.1       0       0      0        0       0              0   
m1_AAACCTGCAGGATCGA.1       0       0      0        1       1              1   
m1_AAACCTGCATCCAACA.1       0       0      0        0       0              0   
m1_AAACCTGTCAGTCCCT.1       1       0      1        0       0              0   
...                       ...     ...    ...      ...     ...            ...   
m2_TTTGTCAGTTGGAGGT.1       1       0      0        0       0              0   
m2_TTTGTCAGTTTGTTTC.1       0       0      1        1       0              0   
m2_TTTGTCATCAACTCTT.1       0       0      1        0       1              0   
m2_TTTGTCATCACGACTA.1       0       0      0        0       0              0   
m2_TTTGTCATCGTGGACC.1       1       1      1        1       0              0   

                       Pcmtd1  Gm26901 

In [27]:
# Find number of genes per sample 
num_genes = converted_matrix.sum(axis = 1)
print(num_genes)

m1_AAACCTGAGAGCAATT.1    2457
m1_AAACCTGCAAGCTGTT.1    1497
m1_AAACCTGCAGGATCGA.1    8026
m1_AAACCTGCATCCAACA.1    1332
m1_AAACCTGTCAGTCCCT.1    3883
                         ... 
m2_TTTGTCAGTTGGAGGT.1    1580
m2_TTTGTCAGTTTGTTTC.1    2908
m2_TTTGTCATCAACTCTT.1    3701
m2_TTTGTCATCACGACTA.1    1320
m2_TTTGTCATCGTGGACC.1    3656
Length: 7097, dtype: int64


In [32]:
percent_mito = pd.DataFrame(converted_matrix.iloc[:,0])
percent_mito.rename(columns={ percent_mito.columns[0]: "percent mito" }, inplace=True)
percent_mito['percent mito'] = 0
print(percent_mito)

                       percent mito
m1_AAACCTGAGAGCAATT.1             0
m1_AAACCTGCAAGCTGTT.1             0
m1_AAACCTGCAGGATCGA.1             0
m1_AAACCTGCATCCAACA.1             0
m1_AAACCTGTCAGTCCCT.1             0
...                             ...
m2_TTTGTCAGTTGGAGGT.1             0
m2_TTTGTCAGTTTGTTTC.1             0
m2_TTTGTCATCAACTCTT.1             0
m2_TTTGTCATCACGACTA.1             0
m2_TTTGTCATCGTGGACC.1             0

[7097 rows x 1 columns]


In [34]:
# Create anndata object


# Set observations x variables data matrix (the dimensions)
adata = ad.AnnData(converted_matrix, dtype="int64")
# print(adata)

# Add gene matrix to obsm (observation/variable level matrices) used for umap embedding
adata.obsm["gene_expression"] = converted_matrix
# print(adata)

# Add n_genes to obs
adata.obs["n_genes"] = num_genes

# Add percent mito to obs
adata.obs["percent_mito"] = percent_mito

# print(adata)

AnnData object with n_obs × n_vars = 7097 × 11713
    obsm: 'gene_expression'
AnnData object with n_obs × n_vars = 7097 × 11713
    obs: 'n_genes', 'percent_mito'
    obsm: 'gene_expression'


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [37]:
# Continue building adata
adata.layers["counts"] = adata.X.copy()

# Normalize data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print(adata)
adata.raw = adata

normalizing counts per cell
    finished (0:00:00)
AnnData object with n_obs × n_vars = 7097 × 11713
    obs: 'n_genes', 'percent_mito'
    uns: 'log1p'
    obsm: 'gene_expression'
    layers: 'counts'


In [45]:
# Feature selection
sc.pp.highly_variable_genes(
    adata, 
    n_top_genes=4000, 
    flavor="seurat_v3",
    batch_key=None, 
    subset=True,
    layer="counts"
)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


In [46]:
scvi.model.TOTALVI.setup_anndata(
    adata, 
    protein_expression_obsm_key="gene_expression",
    layer="counts", 
    batch_key=None
)

INFO     Using column names from columns of adata.obsm['gene_expression']                                          


In [47]:
# Prepare and run model
vae = scvi.model.TOTALVI(adata, latent_distribution="normal")

INFO     Computing empirical prior initialization for protein background.                                          


In [48]:
vae.train()

/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (ElboMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Epoch 194/400:  48%|█████████▋          | 193/400 [44:57<47:42, 13.83s/it, loss=5.52e+04, v_num=1]

ValueError: Expected parameter scale (Tensor of shape (256, 11713)) of distribution Normal(loc: torch.Size([256, 11713]), scale: torch.Size([256, 11713])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[1.4013e-45, 2.8026e-45, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.6048e-22, 9.6073e-22, 9.6889e-25,  ..., 3.9440e-25, 2.1736e-26,
         1.3464e-25],
        [4.8851e-21, 6.1333e-20, 6.3796e-23,  ..., 3.7853e-23, 2.4766e-24,
         1.4997e-23],
        ...,
        [6.9813e-10, 3.0768e-07, 3.8446e-08,  ..., 3.6186e-07, 2.2164e-07,
         3.9703e-07],
        [5.9167e-34, 3.0431e-33, 4.7018e-38,  ..., 6.6850e-39, 1.8168e-41,
         3.4595e-39],
        [4.2039e-45, 1.8217e-44, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])